# Tema 1 AM

##  Server Side

In [ ]:
# importare biblioteci
import socket
import os
import xml.etree.ElementTree as ET
from docx import Document
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
from PIL import Image

In [ ]:
# functiile programului
def receive_docx_from_client(save_path):
    HOST = "127.0.0.1"  # Adresa IP
    PORT = 34512  # Port server

    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server_socket:
        server_socket.bind((HOST, PORT))
        server_socket.listen()
        print("Serverul asteapta conexiuni...")
        conn, addr = server_socket.accept()

        with conn:
            print('Conectat cu', addr)
            # Receive the size of the file
            file_size = int.from_bytes(conn.recv(4), byteorder='big')
            # Receive the file data
            file_data = conn.recv(file_size)
            # Write the file data to disk
            with open(save_path, 'wb') as f:
                f.write(file_data)
            print("Fisier primit cu succes.")

            return save_path

def read_docx_content(file_path):
    doc = Document(file_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

def create_pdf_from_docx(file_path, output_path):
    doc_content = read_docx_content(file_path)
    with open(output_path, 'wb') as f:
        c = canvas.Canvas(f, pagesize=letter)
        page_width, page_height = letter
        margin_left = 100
        margin_bottom = 50
        max_image_width = page_width - 2 * margin_left
        max_image_height = page_height - margin_bottom
        y_coordinate = page_height - margin_bottom
        line_height = 12
        for line in doc_content.split('\n'):
            if 'Table' not in line:
                lines = wrap_text(line, c, max_image_width)
                for wrapped_line in lines:
                    c.drawString(margin_left, y_coordinate, wrapped_line)
                    y_coordinate -= line_height
                    if y_coordinate <= margin_bottom:
                        c.showPage()
                        y_coordinate = page_height - margin_bottom

        # Adaugam imaginea
        img_path = os.path.join("fisiere_server", "image.png")
        save_image_from_docx(file_path, img_path)
        img_width, img_height = fit_image_proportional(img_path, max_image_width, max_image_height)
        c.drawImage(img_path, margin_left, y_coordinate - img_height, width=img_width, height=img_height)

        c.save()

def wrap_text(text, canvas, max_width):
    lines = []
    current_line = ''
    for word in text.split():
        if canvas.stringWidth(current_line + ' ' + word) < max_width:
            current_line += ' ' + word if current_line else word
        else:
            lines.append(current_line.strip())
            current_line = word
    if current_line:
        lines.append(current_line)
    return lines

def save_image_from_docx(file_path, image_file_name="img.png"):
    doc = Document(file_path)
    blip = doc.inline_shapes[0]._inline.graphic.graphicData.pic.blipFill.blip
    rId = blip.embed
    document_part = doc.part
    image_part = document_part.related_parts[rId]
    with open(image_file_name, 'wb') as f:
        f.write(image_part.blob)

def fit_image_proportional(image_path, max_width, max_height):
    img = Image.open(image_path)
    width, height = img.size
    aspect_ratio = width / height

    # Ajuta la pastrarea aspect ratio-ului imaginii
    if width > max_width:
        width = max_width
        height = width / aspect_ratio
    if height > max_height:
        height = max_height
        width = height * aspect_ratio

    return width, height

def extract_table_to_xml(file_path, output_xml_path):
    doc = Document(file_path)
    for table in doc.tables:
        table_xml = ET.Element("table")
        header_row = True  # Flag pentru randul header
        for row in table.rows:
            if header_row:
                header_row = False
                continue
            row_xml = ET.SubElement(table_xml, "row")
            for cell in row.cells:
                cell_xml = ET.SubElement(row_xml, "cell")
                # Am incercat sa adaug si cateva stiluri celulelor (stiu ca xml e in general doar pt date si nestilizat dar citisem ca a fost implementata si aceasta functionalitate si am zis sa incerc)
                cell_xml.set("style", "text-align: center; border: 1px solid black; padding: 5px;")
                cell_xml.text = cell.text
        tree = ET.ElementTree(table_xml)
        # Instructiuni de procesare stylesheet
        tree.write(output_xml_path, encoding="utf-8", xml_declaration=True)

In [ ]:
if __name__ == "__main__":
    # Path-ul unde se salveaza fisierul primit
    docx_save_path = os.path.join("fisiere_server", "received_report.docx")

    # Primim fisierul DOCX de la client
    received_docx_path = receive_docx_from_client(docx_save_path)

    # Facem operatiile pe fisierul DOCX
    create_pdf_from_docx(received_docx_path, os.path.join("fisiere_server", "report_without_table.pdf"))
    extract_table_to_xml(received_docx_path, os.path.join("fisiere_server", "table_data.xml"))

    print("Operatiile de pe server au fost terminate.")